In [2]:
import mhcflurry
from mhcflurry.dataset import Dataset
import os
from collections import defaultdict

Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


In [3]:
bd2009 = Dataset.from_csv("../data/bdata.2009.mhci.public.1.txt")
bd2009

Dataset(n=137654, alleles=['ELA-A1', 'Gogo-B0101', 'H-2-DB', 'H-2-DD', 'H-2-KB', 'H-2-KD', 'H-2-KK', 'H-2-LD', 'HLA-A0101', 'HLA-A0201', 'HLA-A0202', 'HLA-A0203', 'HLA-A0205', 'HLA-A0206', 'HLA-A0207', 'HLA-A0210', 'HLA-A0211', 'HLA-A0212', 'HLA-A0216', 'HLA-A0219', 'HLA-A0250', 'HLA-A0301', 'HLA-A0302', 'HLA-A11', 'HLA-A1101', 'HLA-A2', 'HLA-A2301', 'HLA-A2402', 'HLA-A2403', 'HLA-A2501', 'HLA-A26', 'HLA-A2601', 'HLA-A2602', 'HLA-A2603', 'HLA-A2902', 'HLA-A3001', 'HLA-A3002', 'HLA-A3101', 'HLA-A3201', 'HLA-A3301', 'HLA-A6601', 'HLA-A6801', 'HLA-A6802', 'HLA-A6901', 'HLA-A8001', 'HLA-B0702', 'HLA-B0801', 'HLA-B0802', 'HLA-B0803', 'HLA-B1402', 'HLA-B1501', 'HLA-B1502', 'HLA-B1503', 'HLA-B1509', 'HLA-B1517', 'HLA-B1801', 'HLA-B2701', 'HLA-B2702', 'HLA-B2703', 'HLA-B2705', 'HLA-B3501', 'HLA-B3503', 'HLA-B3508', 'HLA-B3801', 'HLA-B3901', 'HLA-B4001', 'HLA-B4002', 'HLA-B4201', 'HLA-B44', 'HLA-B4402', 'HLA-B4403', 'HLA-B4501', 'HLA-B4601', 'HLA-B4801', 'HLA-B5101', 'HLA-B5301', 'HLA-B5401', '

In [5]:
bd2009_dict = bd2009.to_nested_dictionary()

In [6]:
bd2009_dict.keys()

dict_keys(['HLA-B5101', 'H-2-KD', 'HLA-A3002', 'HLA-A0210', 'Patr-B0101', 'HLA-A0206', 'Mamu-B17', 'HLA-A0203', 'HLA-B2702', 'HLA-A8001', 'HLA-B4002', 'H-2-DD', 'HLA-A1101', 'HLA-A0201', 'H-2-LD', 'HLA-B4501', 'HLA-A0205', 'HLA-A0202', 'HLA-A0212', 'HLA-B5301', 'HLA-A6901', 'HLA-B0802', 'HLA-B5802', 'HLA-A11', 'HLA-B1402', 'HLA-A2', 'HLA-A0207', 'H-2-KB', 'HLA-B3508', 'HLA-A6801', 'HLA-A0302', 'Patr-A0602', 'HLA-A0301', 'HLA-A3101', 'HLA-A0216', 'Mamu-A02', 'HLA-B4001', 'Patr-B0901', 'HLA-B1509', 'HLA-B0801', 'HLA-A2601', 'HLA-B4402', 'HLA-A26', 'Patr-B2401', 'HLA-B3501', 'HLA-A0219', 'HLA-A2301', 'HLA-B2705', 'HLA-A0211', 'HLA-B2703', 'Mamu-A2201', 'HLA-B1503', 'HLA-B4601', 'ELA-A1', 'HLA-B4801', 'HLA-A2403', 'HLA-B4403', 'Patr-B1301', 'Mamu-B08', 'HLA-B1501', 'HLA-B3503', 'HLA-B0702', 'Mamu-A2601', 'HLA-A2902', 'HLA-A6802', 'Mamu-A07', 'HLA-A0101', 'HLA-B44', 'HLA-A2602', 'Patr-A0901', 'HLA-B3801', 'HLA-E0101', 'Mamu-B03', 'H-2-DB', 'HLA-B4201', 'HLA-A3301', 'HLA-B5701', 'HLA-A2603',

In [47]:
from mhcflurry.imputation_helpers import imputer_from_name
from fancyimpute import MICE

imputation_algos = {
    name: imputer_from_name(name, min_value=0)
    for name in ["knn", "softimpute", "svd"]
}

imputation_algos["mice-burnin5-iters25"] = MICE(
    n_burn_in=5,
    n_imputations=25)

imputation_algos["mice-burnin10-iters25"] = MICE(
    n_burn_in=10,
    n_imputations=25)


imputation_algos["mice-burnin10-iters50"] = MICE(
    n_burn_in=10,
    n_imputations=50)

imputation_algos["mice-sparse5-burnin5-iters25"] = MICE(
    n_burn_in=5,
    n_imputations=25,
    n_nearest_columns=5)

imputation_algos["mice-sparse25-burnin5-iters25"] = MICE(
    n_burn_in=5,
    n_imputations=25,
    n_nearest_columns=25)

imputation_algos["mice-sparse5-burnin5-iters50"] = MICE(
    n_burn_in=5,
    n_imputations=50,
    n_nearest_columns=5)


imputation_algos["mice-sparse25-burnin5-iters50"] = MICE(
    n_burn_in=5,
    n_imputations=50,
    n_nearest_columns=25)


imputation_algos["mice-sparse25-burnin5-iters50"] = MICE(
    n_burn_in=5,
    n_imputations=50,
    n_nearest_columns=25)



In [48]:
from sklearn.metrics import f1_score, roc_auc_score 
from scipy.stats import kendalltau

kendalltau([0.908, 0.2, -3], [0.38, -1, 0.4])

KendalltauResult(correlation=-0.33333333333333326, pvalue=0.60150814411369402)

In [49]:

def evaluate_imputed_ic50s(imputed_dataset, test_dataset, binding_cutoff=500.0, filename=None):
    """
    Keep only pMHCs contained in both datasets, returns DataFrame with the following columns:
        - allele
        - peptide
        - test_ic50
        - imputed_ic50
    """
    imputed_dict = imputed_dataset.to_nested_dictionary()
    test_ic50s = []
    imputed_ic50s = []
    for (allele, peptide, ic50) in zip(test_dataset.alleles, test_dataset.peptides, test_dataset.affinities):
        allele_dict = imputed_dict.get(allele, {})
        if peptide in allele_dict:
            imputed_ic50 = allele_dict[peptide]
        else:
            continue
        test_ic50s.append(ic50)
        imputed_ic50s.append(imputed_ic50)
    test_ic50s = np.asarray(test_ic50s)
    imputed_ic50s = np.asarray(imputed_ic50s)
    print("Keeping %d of %d imputed values, %d test values" % (
        len(test_ic50s),
        len(imputed_dataset),
        len(test_dataset)))
    return dict(
        f1=f1_score(y_true=test_ic50s<=binding_cutoff, y_pred=imputed_ic50s<=binding_cutoff),
        auc=roc_auc_score(y_true=test_ic50s<=binding_cutoff, y_score=-imputed_ic50s),
        tau=kendalltau(test_ic50s, imputed_ic50s).correlation
    )

def evaluate_imputation_method(imputer, train, test):
    """
    Returns dictionary with "auc", "f1", "tau" keys.
    """
    train_imputed = train.impute_missing_values(
        imputation_method=imputer,
        min_observations_per_peptide=2,
        min_observations_per_allele=2)
    return evaluate_imputed_ic50s(imputed_dataset=train_imputed, test_dataset=test)

def scores_generator(all_data, imputation_algos, n_folds=3):
    """
    Generator yields (imputer_name, cv_fold_idx, scores_dict)
    """
    for i, (train, test) in enumerate(all_data.cross_validation_iterator(n_folds=n_folds)):
        print("=== CV fold %d/%d" % (i + 1, n_folds))
        for name, imputer in imputation_algos.items():
            print("--> Running imputation algorithm %s" % name)
            scores = evaluate_imputation_method(imputer, train, test)
            print(scores)
            yield (name, i, scores)

def evaluate_imputation_methods(filename, all_data, imputation_algos, n_folds=3):
    """
    Writes out results to filename and also returns dictionary from algorithms to score collections.
    """
    f = open(filename, "w")
    f.write("imputation_algorithm,cv_fold,f1,tau,auc\n")
    results = defaultdict(list)
    for name, cv_fold_idx, scores in scores_generator(all_data, imputation_algos, n_folds):
        line = "%s,%d,%f,%f,%f\n" % (name, cv_fold_idx, scores["f1"], scores["tau"], scores["auc"])
        print(line)
        f.write(line)
        f.flush()
        results[name].append(scores)
    f.close()
    return results

In [50]:
results = evaluate_imputation_methods("Compare-Imputation.csv", bd2009, imputation_algos)

=== CV fold 1/3
--> Running imputation algorithm mice-burnin5-iters25
Dropping 11113 peptides with <2 observations
Dropping 9 alleles with <2 observations: ['ELA-A1', 'HLA-B2701', 'HLA-B3508', 'HLA-B4201', 'HLA-B44', 'Mamu-A07', 'Mamu-B04', 'Patr-A0602', 'Patr-B1701']
[MICE] Completing matrix with shape (15655, 95)
[MICE] Starting imputation round 1/30, elapsed time 0.047
[MICE] Starting imputation round 2/30, elapsed time 3.708
[MICE] Starting imputation round 3/30, elapsed time 7.330
[MICE] Starting imputation round 4/30, elapsed time 11.206
[MICE] Starting imputation round 5/30, elapsed time 14.837
[MICE] Starting imputation round 6/30, elapsed time 18.625
[MICE] Starting imputation round 7/30, elapsed time 22.519
[MICE] Starting imputation round 8/30, elapsed time 26.480
[MICE] Starting imputation round 9/30, elapsed time 30.414
[MICE] Starting imputation round 10/30, elapsed time 34.524
[MICE] Starting imputation round 11/30, elapsed time 38.387
[MICE] Starting imputation round 12

In [54]:
score_names = ["f1", "tau", "auc"]

In [57]:
df = pd.read_csv("Compare-Imputation.csv")

In [59]:
df["combined-score"] = df["f1"] + df["tau"] + 2 * (df["auc"] - 0.5)

In [66]:
df.groupby("imputation_algorithm").mean().sort_values(by="combined-score")

,cv_fold,f1,tau,auc,combined-score
imputation_algorithm,,,,,
mice-sparse5-burnin5-iters25,1,0.491753,0.385920,0.819017,1.515707
mice-sparse5-burnin5-iters50,1,0.499054,0.413002,0.832796,1.577647
knn,1,0.567644,0.418689,0.810401,1.607136
svd,1,0.596698,0.407354,0.831474,1.667000
mice-burnin10-iters25,1,0.564432,0.437443,0.844214,1.690304
mice-burnin5-iters25,1,0.561710,0.439339,0.845651,1.692352
mice-burnin10-iters50,1,0.567713,0.454074,0.851116,1.724020
softimpute,1,0.602133,0.430919,0.845749,1.724550
mice-sparse25-burnin5-iters25,1,0.574720,0.449712,0.852053,1.728538
